In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def Binary_Call(S_t,K,T,r,sigma,D):
    
    d1 = (np.log(S_t / K) + ((r - D) + (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))  
    d2 = (np.log(S_t / K) + ((r - D) - (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))
    
    
    return np.exp(-r * T) * norm.cdf(d2)

In [3]:
def Binary_Put(S_t,K,T,r,sigma,D):
    
    d1 = (np.log(S_t / K) + ((r - D) + (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))  
    d2 = (np.log(S_t / K) + ((r - D) - (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))
    
    return np.exp(-r * T) * norm.cdf(-d2)

In [4]:
def Regular_Bin_DIC(S_t,K,T,r,sigma,D,B):
    
    d1 = (np.log(B /((K * S_t)/B)) + ((r - D) + (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))
    d2 = (np.log(B /((K * S_t)/B)) + ((r - D) - (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))
    
    
    gamma = 1 - 2 * ((r - D)/(sigma ** 2))
    
    
    return np.power((S_t / B), gamma) * Binary_Call(B,((K * S_t)/B),T,r,sigma,D)

In [5]:
def Reverse_Bin_DIC(S_t,K,T,r,sigma,D,B) : 
    
    d1 = (np.log(B /((K * S_t)/B)) + ((r - D) + (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))
    d2 = (np.log(B /((K * S_t)/B)) + ((r - D) - (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))
    
    
    return Binary_Call(S_t,K,T,r,sigma,D) - np.exp(-r * T) + Binary_Put(S_t,K,T,r,sigma,D) + Regular_Bin_DIC(S_t,B,T,r,sigma,D,B)

In [6]:
def Call(S_t,K,T,r,sigma,D):
    
    d1 = (np.log(S_t / K) + ((r - D) + (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))  
    d2 = (np.log(S_t / K) + ((r - D) - (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))
    
    
    return S_t * np.exp(-D * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [7]:
 def Regular_DIC(S_t,K,T,r,sigma,D,B):
        
        d1 = (np.log(B /((K * S_t)/B)) + ((r - D) + (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))
        d2 = (np.log(B /((K * S_t)/B)) + ((r - D) - (np.power(sigma,2)/ 2) * T )) / (sigma * np.sqrt(T))
        
        
        if (r == D):
            
            if (S_t < B or S_t == B):
                
                return Call(S_t,K,T,r,sigma,D)
            
            else:
                
                return B * np.exp(-D * T) * norm.cdf(d1) - ((K * S_t)/B) * np.exp(-r * T) * norm.cdf(d2)
            
        else :
            
            if (S_t < B or S_t == B):
                
                return Call(S_t,K,T,r,sigma,D)
            
            
            else : 
                
                temp = B * np.exp(-D * T) * norm.cdf(d1) - ((K * S_t)/B) * np.exp(-r * T) * norm.cdf(d2)
                
                gamma = 1 - 2 * ((r - D)/(sigma ** 2))
                
                return np.power((S_t / B), (gamma - 1)) * temp 

In [8]:
def Reverse_DIC(S_t,K,T,r,sigma,D,B):
    
    return Call(S_t,K,T,r,sigma,D) - Call(S_t,B,T,r,sigma,D) - (D - K) * Binary_Call(S_t,B,T,r,sigma,D) + Regular_DIC(S_t,B,T,r,sigma,D,B) + (D - K) * Regular_Bin_DIC(S_t,B,T,r,sigma,D,B)

In [9]:
def Reverse_DOC(S_t,K,T,r,sigma,D,B):
    
    return Call(S_t,K,T,r,sigma,D) - Reverse_DIC(S_t,K,T,r,sigma,D,B)

In [10]:
import math
def combos(n, i):
    return math.factorial(n) / (math.factorial(n-i)*math.factorial(i))

In [52]:
N = 100
S0  = 100
T = 1
sigma = 0.2
K = 105
r = 0.05

def Continous_Binomial_Option_Pricing(S0, K , T, r, sigma, N, type_ = 'call'):
    dt = T/N
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))
    p = (  np.exp(r*dt) - d )  /  (  u - d )
    value = 0 
    for i in range(N+1):
        node_prob = combos(N, i)*p**i*(1-p)**(N-i)
        ST = S0*(u)**i*(d)**(N-i)
        if type_ == 'call':
            value += max(ST-K,0) * node_prob
        elif type_ == 'put':
            value += max(K-ST, 0)*node_prob
        else:
            raise ValueError("type_ must be 'call' or 'put'" )
    
    return value*np.exp(-r*T)

In [53]:
def ExchangeOption(X_t,Y_t,D_x,D_y,sigma_x,sigma_y,rho_xy,T):
    
    import numpy as np
    from scipy.stats import norm
    
    sigma = np.sqrt(np.power(sigma_x,2) - 2 * rho_xy * sigma_x * sigma_x + np.power(sigma_y,2))
    
    d_1 = (np.log(X_t/Y_t) + ((D_y - D_x + ((1/2) * np.power(sigma,2))) * (T))) / (sigma * np.sqrt(T))
    
    d_2 = (np.log(X_t/Y_t) + ((D_y - D_x - ((1/2) * np.power(sigma,2))) * (T))) / (sigma * np.sqrt(T))
    
    
    return X_t * np.exp(-D_x * T) * norm.cdf(d_1) - Y_t * np.exp(-D_y * T) * norm.cdf(d_2)

In [54]:
N = 2
S0  = 100.820000
T = 1
K = 100.820000
r = 0.005000

def Continous_Binomial_Option_Pricing(S0, K , T, r, N, type_ = 'call'):
    dt = T/N
    u = 1.600000
    d = 1.710000
    p = (  np.exp(r*dt) - d )  /  (  u - d )
    value = 0 
    for i in range(N+1):
        node_prob = combos(N, i)*p**i*(1-p)**(N-i)
        ST = S0*(u)**i*(d)**(N-i)
        if type_ == 'call':
            value += max(ST-K,0) * node_prob
        elif type_ == 'put':
            value += max(K-ST, 0)*node_prob
        else:
            raise ValueError("type_ must be 'call' or 'put'" )
    
    return value*np.exp(-r*T)

In [55]:
Continous_Binomial_Option_Pricing(S0,K,T,r,N,'call')

0.5028418477928512

In [76]:
import math

def binomial_model(S, K, r, T, sigma, N, option_type):
    """
    Fonction pour calculer le prix d'une option européenne avec le modèle binomial

    Arguments :
    S : float, prix actuel du sous-jacent
    K : float, prix d'exercice de l'option
    r : float, taux d'intérêt sans risque
    T : float, durée jusqu'à l'échéance de l'option (en années)
    sigma : float, volatilité du sous-jacent
    N : int, nombre de périodes de l'arbre binomial
    option_type : str, "call" pour une option d'achat, "put" pour une option de vente

    Retour :
    float, le prix de l'option
    """
    dt = T / N  # taille d'un intervalle de temps
    u = math.exp(sigma * math.sqrt(dt))  # facteur d'augmentation
    d = 1 / u  # facteur de diminution
    p = (math.exp(r * dt) - d) / (u - d)  # probabilité risque neutre

    # Calcul des prix finaux de l'option
    stock_prices = []
    if option_type == "call":
        for i in range(N+1):
            stock_prices.append(max(S * u**(N-i) * d**i - K, 0))
    elif option_type == "put":
        for i in range(N+1):
            stock_prices.append(max(K - S * u**(N-i) * d**i, 0))

    # Calcul des prix à chaque étape de l'arbre
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            stock_prices[j] = math.exp(-r*dt) * (p*stock_prices[j] + (1-p)*stock_prices[j+1])

    return stock_prices[0]

# Exemple d'utilisation :
S = 100  # prix actuel du sous-jacent
K = 105  # prix d'exercice de l'option
r = 0.05  # taux d'intérêt sans risque
T = 1  # durée jusqu'à l'échéance de l'option (en années)
sigma = 0.2  # volatilité du sous-jacent
N = 100  # nombre de périodes de l'arbre binomial
option_type = "call"  # type d'option (call ou put)

option_price = binomial_model(S, K, r, T, sigma, N, option_type)

print("Le prix de l'option est :", option_price)

Le prix de l'option est : 8.026229025058432
